# Multiple models with Pipeline and ColumnTransformer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = sns.load_dataset("tips")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
#predictthe total_bill? total_bill is target variables
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [ ]:
#EDA>> subjective
#encoding, missing value treatiment, scaling>> automate
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [ ]:
X = df.drop('total_bill', axis =1)
y = df['total_bill']

In [ ]:
X

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
1,1.66,Male,No,Sun,Dinner,3
2,3.50,Male,No,Sun,Dinner,3
3,3.31,Male,No,Sun,Dinner,2
4,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,5.92,Male,No,Sat,Dinner,3
240,2.00,Female,Yes,Sat,Dinner,2
241,2.00,Male,Yes,Sat,Dinner,2
242,1.75,Male,No,Sat,Dinner,2


In [ ]:
y

0      16.99
1      10.34
2      21.01
3      23.68
4      24.59
       ...  
239    29.03
240    27.18
241    22.67
242    17.82
243    18.78
Name: total_bill, Length: 244, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [ ]:
X_train.head()

,tip,sex,smoker,day,time,size
0,1.01,Female,No,Sun,Dinner,2
154,2.00,Male,No,Sun,Dinner,4
167,4.50,Male,No,Sun,Dinner,4
110,3.00,Male,No,Sat,Dinner,2
225,2.50,Female,Yes,Fri,Lunch,2


In [ ]:
df.isna().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [ ]:
#Handling the missing value
#data encoding
#feature scaling

from sklearn.impute import SimpleImputer #for missing vlaue
from sklearn.preprocessing import OneHotEncoder #for encoding
from sklearn.preprocessing import StandardScaler #for scaling

from sklearn.pipeline import Pipeline #A sequence of data transformers
from sklearn.compose import ColumnTransformer #Groups all the pipeline steps for each of the clumns

In [ ]:
cat_cols = ["sex", "smoker", "day", "time"]
num_cols = ["tip", "size"]

In [ ]:
#feature engineering automation using pipeline and columntransformer

num_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = "median")),
                                ('scaling', StandardScaler())])
cat_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = "most_frequent")),
                                ('encoding', OneHotEncoder())])

In [ ]:
preprocessor = ColumnTransformer([("num_pipeline", num_pipeline, num_cols),
                  ("cat_pipeline", cat_pipeline, cat_cols)])

In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
X_train

array([[-1.47443803, -0.57766863,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [-0.71612531,  1.47042924,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.19880579,  1.47042924,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.43283335, -0.57766863,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.29060464, -0.57766863,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.1034652 ,  0.44638031,  1.        , ...,  0.        ,
         1.        ,  0.        ]])

In [ ]:
# X_test

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
models = {"support vector Regressor": SVR(),
         "DT Regressor": DecisionTreeRegressor(),
          "Multiple Linear regression": LinearRegression(),
         "Random_forest_Regressor": RandomForestRegressor()}

In [ ]:
from sklearn.metrics import r2_score
def model_train_eval(X_train, y_train, X_test, y_test, models):
    evaluation = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        model_score = r2_score(y_test, y_pred)
        evaluation[list(models.keys())[i]] = model_score
    return evaluation

In [ ]:
model_train_eval(X_train, y_train, X_test, y_test, models)

{'support vector Regressor': 0.4069433055293954,
 'DT Regressor': 0.148273719426101,
 'Multiple Linear regression': 0.6719006761625261,
 'Random_forest_Regressor': 0.6344196825893511}

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(oob_score=True)

In [ ]:
rfr

RandomForestRegressor(oob_score=True)

In [ ]:
# X_train, X_test

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
params={'max_depth':[1,50, 100, 150,200],
        'n_estimators':[50, 100,200]}

In [ ]:
params

{'max_depth': [1, 50, 100, 150, 200], 'n_estimators': [50, 100, 200]}

In [ ]:
reg = RandomizedSearchCV(rfr, param_distributions=params, cv = 5, verbose =3, scoring = 'r2')

In [ ]:
reg

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   param_distributions={'max_depth': [1, 50, 100, 150, 200],
                                        'n_estimators': [50, 100, 200]},
                   scoring='r2', verbose=3)

In [ ]:
reg.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .....max_depth=1, n_estimators=100;, score=0.234 total time=   0.2s
[CV 2/5] END .....max_depth=1, n_estimators=100;, score=0.249 total time=   0.2s
[CV 3/5] END .....max_depth=1, n_estimators=100;, score=0.437 total time=   0.2s
[CV 4/5] END .....max_depth=1, n_estimators=100;, score=0.485 total time=   0.2s
[CV 5/5] END .....max_depth=1, n_estimators=100;, score=0.399 total time=   0.2s
[CV 1/5] END ....max_depth=100, n_estimators=50;, score=0.278 total time=   0.1s
[CV 2/5] END ....max_depth=100, n_estimators=50;, score=0.464 total time=   0.1s
[CV 3/5] END ....max_depth=100, n_estimators=50;, score=0.225 total time=   0.1s
[CV 4/5] END ....max_depth=100, n_estimators=50;, score=0.545 total time=   0.1s
[CV 5/5] END ....max_depth=100, n_estimators=50;, score=0.356 total time=   0.1s
[CV 1/5] END ...max_depth=200, n_estimators=200;, score=0.289 total time=   0.5s
[CV 2/5] END ...max_depth=200, n_estimators=200;

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   param_distributions={'max_depth': [1, 50, 100, 150, 200],
                                        'n_estimators': [50, 100, 200]},
                   scoring='r2', verbose=3)

In [ ]:
reg.best_params_

{'n_estimators': 100, 'max_depth': 200}

In [ ]:
reg.best_score_

0.4106241440107194

# Random Forest Classifer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = sns.load_dataset("tips")
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [ ]:
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [ ]:
#EDA
#ENCODING, SCALING, MISSING

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df["time"])
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [ ]:
#aim>>to predict time

In [ ]:
df.time.unique()#dinner>0, lunch>1

array([0, 1])

In [ ]:
X = df.drop('time', axis =1)
y = df['time']

In [ ]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [ ]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [ ]:
X_train

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
154,19.77,2.00,Male,No,Sun,4
167,31.71,4.50,Male,No,Sun,4
110,14.00,3.00,Male,No,Sat,2
225,16.27,2.50,Female,Yes,Fri,2
...,...,...,...,...,...,...
137,14.15,2.00,Female,No,Thur,2
72,26.86,3.14,Female,Yes,Sat,2
140,17.47,3.50,Female,No,Thur,2
235,10.07,1.25,Male,No,Sat,2


In [ ]:
X_test

,total_bill,tip,sex,smoker,day,size
67,3.07,1.00,Female,Yes,Sat,1
243,18.78,3.00,Female,No,Thur,2
206,26.59,3.41,Male,Yes,Sat,3
122,14.26,2.50,Male,No,Thur,2
89,21.16,3.00,Male,No,Thur,2
218,7.74,1.44,Male,Yes,Sat,2
58,11.24,1.76,Male,Yes,Sat,2
186,20.90,3.50,Female,Yes,Sun,3
177,14.48,2.00,Male,Yes,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [ ]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [ ]:
#handling missing values
#data encoding
#feature scaling

from sklearn.impute import SimpleImputer #for missing values
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


#pipeline>> a sequence of data transformation
#Column transformer>> groups all the pipeline steps for each of the columns


from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [ ]:
cat_cols = ["sex", "smoker", "day"]
num_cols = ["total_bill", "tip", "size"]

In [ ]:
#feature engineering automation using pipeline and column transformer

num_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy = "median")),
                                 ('scaling', StandardScaler())])
cat_pipeline = Pipeline(steps = [("imputation", SimpleImputer(strategy = "most_frequent")),
                                ('encoding', OneHotEncoder())])

In [ ]:
preprocessor = ColumnTransformer([("num_pipeline", num_pipeline, num_cols),
                  ("cat_pipeline", cat_pipeline, cat_cols)])

In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
X_train

array([[-0.28611937, -1.47443803, -0.57766863, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.02695905, -0.71612531,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.3716196 ,  1.19880579,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.23206267,  0.43283335, -0.57766863, ...,  0.        ,
         0.        ,  1.        ],
       [-1.06543688, -1.29060464, -0.57766863, ...,  1.        ,
         0.        ,  0.        ],
       [-0.29287646,  0.1034652 ,  0.44638031, ...,  1.        ,
         0.        ,  0.        ]])

In [ ]:
X_test

array([[-1.85376383, -1.48209775, -1.60171757,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.08453291,  0.04984713, -0.57766863,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.79501474,  0.36389583,  0.44638031,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.59356688, -0.33313909, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.18349826,  0.04984713, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-1.32783714, -1.14506988, -0.57766863,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.   

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


models = {"SVC": SVC(),
         "dtc": DecisionTreeClassifier(),
         "Logreg": LogisticRegression(),
         "rf": RandomForestClassifier()}


In [ ]:
models.values()

dict_values([SVC(), DecisionTreeClassifier(), LogisticRegression(), RandomForestClassifier()])

In [ ]:
from sklearn.metrics import accuracy_score

def model_train_eval(X_train, y_train, X_test, y_test, models):
    evaluation = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        model_score = accuracy_score(y_test, y_pred)
        evaluation[list(models.keys())[i]] = model_score
    return evaluation

In [ ]:
model_train_eval(X_train, y_train, X_test, y_test, models)

{'SVC': 0.9183673469387755,
 'dtc': 0.8979591836734694,
 'Logreg': 0.9183673469387755,
 'rf': 0.8979591836734694}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [ ]:
rf

RandomForestClassifier()

In [ ]:
rf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.8979591836734694

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth' : [1, 2, 3,4, 10, 20, 30],
         'n_estimators': [50, 100, 200, 300, 400, 1000],
         'criterion': ["gini", 'entropy']}

In [ ]:
params

{'max_depth': [1, 2, 3, 4, 10, 20, 30],
 'n_estimators': [50, 100, 200, 300, 400, 1000],
 'criterion': ['gini', 'entropy']}

In [ ]:
clf = GridSearchCV(rf, param_grid=params, cv = 5, scoring = "accuracy", verbose=3)

In [ ]:
clf.fit(X_train, y_train)

Fitting 5 folds for each of 84 candidates, totalling 420 fits
[CV 1/5] END criterion=gini, max_depth=1, n_estimators=50;, score=0.795 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=1, n_estimators=50;, score=0.949 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=1, n_estimators=50;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=1, n_estimators=50;, score=1.000 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=1, n_estimators=50;, score=1.000 total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=1, n_estimators=100;, score=0.744 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=1, n_estimators=100;, score=0.974 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=1, n_estimators=100;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=1, n_estimators=100;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=1, n_estimators=100;, score=0.744 total time=   0.2s
[CV 1/5] END criterion=

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4, 10, 20, 30],
                         'n_estimators': [50, 100, 200, 300, 400, 1000]},
             scoring='accuracy', verbose=3)

In [ ]:
clf.best_params_

{'criterion': 'gini', 'max_depth': 1, 'n_estimators': 300}

In [ ]:
clf.best_score_

0.9794871794871796

# OOB Score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
X, y = make_classification(n_samples = 1000, n_features = 30, n_classes = 2, random_state=1)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=1)

In [ ]:
rf.fit(X, y)

RandomForestClassifier(oob_score=True, random_state=1)

In [ ]:
rf.oob_score_

0.86

In [ ]:
#internal homework: